In [ ]:
%pip install polars
%pip install pyarrow
%pip install duckdb
%pip install panadas

  Using cached polars-1.29.0-cp39-abi3-win_amd64.whl.metadata (15 kB)
Using cached polars-1.29.0-cp39-abi3-win_amd64.whl (35.0 MB)
Note: you may need to restart the kernel to use updated packages.


In [17]:
import pandas as pd
import polars as pl # type: ignore
import duckdb

In [ ]:
duckdb.sql("""
COPY (
    SELECT 
        product_name, 
        generic_name, 
        categories, 
        categories_tags, 
        countries_tags, 
        labels_tags, 
        ingredients_tags, 
        nutriscore_grade, 
        nova_group, 
        ecoscore_grade, 
        scans_n 
    FROM '../data/food.parquet'
    WHERE array_contains(countries_tags, 'en:united-states')
) TO '../data/food_us.csv' (HEADER, FORMAT 'csv')
""")

#Check the size of the parquet and csv files
parquet_size = duckdb.sql("SELECT COUNT(*) FROM '../data/food.parquet'").to_df().iloc[0, 0]
csv_size = duckdb.sql("SELECT COUNT(*) FROM '../data/food_us.csv'").to_df().iloc[0, 0]
print(f"Parquet size: {parquet_size}")
print(f"CSV size: {csv_size}")